# Описание

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В
свободное время жители городов больше не выходят на улицу, не посещают кафе
и торговые центры, предпочитают провест время за чтением книги. 

Для компании, которой принадлежит крупный сервис для чтения книг по подписке, необходимо провести исследование. Результаты аналитической работы позволят компании сформулировать ценностное предложение для нового продукта.

**Цель исследования:** — проанализировать базу данных сервиса для чтения книг по подписке.

В базе данных хранится информация о книгах, издательствах, авторах, а также пользовательские
обзоры книг. 

**Описание данных**

**Таблица** `books`

Содержит данные о книгах:

`book_id` — идентификатор книги;

`author_id` — идентификатор автора;

`title` — название книги;

`num_pages` — количество страниц;

`publication_date` — дата публикации книги;

`publisher_id` — идентификатор издателя.

**Таблица** `authors`


Содержит данные об авторах:

`author_id` — идентификатор автора;

`author` — имя автора.

**Таблица** `publishers`

Содержит данные об издательствах:

`publisher_id` — идентификатор издательства;

`publisher` — название издательства;

**Таблица** `ratings`

Содержит данные о пользовательских оценках книг:

`rating_id` — идентификатор оценки;

`book_id` — идентификатор книги;

`username` — имя пользователя, оставившего оценку;

`rating` — оценка книги.

**Таблица** `reviews`

Содержит данные о пользовательских обзорах на книги:

`review_id` — идентификатор обзора;

`book_id` — идентификатор книги;

`username` — имя пользователя, написавшего обзор;

`text` — текст обзора.

In [4]:
from IPython.display import Image
display(Image(url='https://pictures.s3.yandex.net/resources/scheme_1589269096.png', 
              width = 700) 
       ) 


# Шаг 1. Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# Шаг 2. Получение информации из базы данных

**Узнаем, сколько книг вышло после 1 января 2000 года**

In [3]:
query = ''' SELECT COUNT (book_id)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df.head(5))


   count
0    819


**В базе данных хранится 819 книг, вышедших с 1 января 2000 года**

**Найдем для каждой книги количество обзоров и среднюю оценку**

In [4]:
query = ''' SELECT b.title,
                   AVG(r.rating),       
                   COUNT(rw.review_id)
            FROM books AS b
            LEFT JOIN ratings AS r ON b.book_id = r.book_id
            LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
            GROUP BY b.title
            ORDER BY AVG(r.rating) DESC;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

                                                 title   avg  count
0    Captivating: Unveiling the Mystery of a Woman'...  5.00      4
1                                        Evening Class  5.00      4
2    In the Hand of the Goddess (Song of the Liones...  5.00      6
3                   The Big Bad Wolf  (Alex Cross  #9)  5.00      4
4                        A Dirty Job (Grim Reaper  #1)  5.00     16
..                                                 ...   ...    ...
994  The World Is Flat: A Brief History of the Twen...  2.25     12
995                  His Excellency: George Washington  2.00      4
996                                              Junky  2.00      4
997                                      Drowning Ruth  2.00      9
998                               Harvesting the Heart  1.50      4

[999 rows x 3 columns]


**Узнаем сколько книг в базах данных**

In [5]:
query = ''' SELECT COUNT(DISTINCT title)
            FROM books;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

   count
0    999


In [6]:
query = ''' SELECT COUNT(DISTINCT book_id)
            FROM books;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

   count
0   1000


**Уникальных названий 999, а всего 1000 книг**

In [7]:
query = ''' SELECT b.title,
                   AVG(r.rating),       
                   COUNT(DISTINCT rw.review_id)
            FROM books AS b
            LEFT JOIN ratings AS r ON b.book_id = r.book_id
            LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
            GROUP BY b.book_id
            ORDER BY AVG(r.rating) DESC;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

                                                 title   avg  count
0        Arrows of the Queen (Heralds of Valdemar  #1)  5.00      2
1    The Walking Dead  Book One (The Walking Dead #...  5.00      2
2                                      Light in August  5.00      2
3    Wherever You Go  There You Are: Mindfulness Me...  5.00      2
4    Captivating: Unveiling the Mystery of a Woman'...  5.00      2
..                                                 ...   ...    ...
995  The World Is Flat: A Brief History of the Twen...  2.25      3
996                  His Excellency: George Washington  2.00      2
997                                      Drowning Ruth  2.00      3
998                                              Junky  2.00      2
999                               Harvesting the Heart  1.50      2

[1000 rows x 3 columns]


**На основе этих данных пользователям приложения можно предлагать книги, например, с наивысшим рейтингом**

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [8]:
query = ''' SELECT p.publisher     
            FROM publishers AS p
            LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

       publisher
0  Penguin Books


**Согласно полученным данным Penguin Books издает наибольшее количество книг**

**Определим автора с самой высокой средней оценкой книг (учитены книги с 50 и более оценками)**

In [9]:
query = ''' SELECT a.author
            FROM authors AS a
            LEFT JOIN books AS b ON a.author_id = b.author_id
            WHERE book_id IN (SELECT book_id                      
                              FROM ratings   
                              GROUP BY book_id
                              HAVING COUNT(rating) >= 50
                              ORDER BY AVG(rating) DESC
                              LIMIT 1);
                 
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

                       author
0  J.K. Rowling/Mary GrandPré


**Книги Джоан Роулинг имеют самую высокую среднюю оценку. Данную информацию можно также учитывать при разработке нового продукта приложения для читателей**

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок**


In [10]:
query = ''' SELECT AVG(review)
            FROM
            (SELECT username,
                   COUNT(review_id) AS review
            FROM reviews
            WHERE username IN(SELECT username
                              FROM ratings
                              GROUP BY username 
                              HAVING COUNT(rating) > 50)
            GROUP BY username) AS i;
        '''

books_df = pd.io.sql.read_sql(query, con=engine)

print(books_df)

         avg
0  24.333333


**В среднем пользователи, которые оставили больше 50 оценок, пишут по 24 отзыва.**

## Общий вывод:

В результате анализа баз данных удалось получить следующую информацию:
- в базе данных хранится 819 книг, вышедших с 1 января 2000 года;
- нашли для каждой книги количество обзоров и среднюю оценку;
- издательство Penguin Books выпустило наибольшее число книг толще 50 страниц;
- Джоан Роулинг является автором с самой высокой средней оценкой книг (учитены книги с 50 и более оценками);
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24.3

На основе полученных данных пользователям приложения можно рекомендовать книги, авторов, издания книг. Также можно выявлять активных пользователей. 